In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
files.upload()

In [ ]:
!cp /content/drive/MyDrive/train.txt /content/train.txt 
!cp /content/drive/MyDrive/val.txt /content/val.txt 
!cp /content/drive/MyDrive/train.cache /content/train.cache 
!cp /content/drive/MyDrive/val.cache /content/val.cache 
!cp /content/drive/MyDrive/kaggle.json /content/kaggle.json 

In [ ]:
!mkdir '/root/.kaggle'
!cp kaggle.json '/root/.kaggle'
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
 !pip install -q kaggle

In [ ]:
pwd

'/content'

In [ ]:
!kaggle datasets download -d awsaf49/vinbigdata-1024-image-dataset

100% 8.14G/8.14G [03:08<00:00, 42.2MB/s]
100% 8.14G/8.14G [03:08<00:00, 46.3MB/s]


In [ ]:
!unzip -q vinbigdata-1024-image-dataset.zip

In [ ]:
cd /content/drive/MyDrive/xray100

/content/drive/MyDrive/xray100


In [ ]:
labels = [
    "Aortic enlargement",
    "Atelectasis",
    "Calcification",
    "Cardiomegaly",
    "Consolidation",
    "ILD",
    "Infiltration",
    "Lung Opacity",
    "Nodule/Mass",
    "Other lesion",
    "Pleural effusion",
    "Pleural thickening",
    "Pneumothorax",
    "Pulmonary fibrosis",
    # "No finding",
    # "Finding"
]

In [ ]:
import pandas as pd
df_train = pd.read_csv('/content/drive/MyDrive/xray100/vinbigdata/train.csv')
df_train = df_train[df_train.class_id!=14].reset_index(drop = True)


In [ ]:
import pandas as pd
df_train = pd.read_csv('/content/vinbigdata/train.csv')
df_train = df_train[df_train.class_id!=14].reset_index(drop = True)


In [ ]:
def bb_iou(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou
 
 
def averageCoordinates(df, threshold):
    tmp_df = df.reset_index()
    duplicate = {}
    for index1, row1 in tmp_df.iterrows():
        if index1 < len(tmp_df) - 1:
            next_index = index1 + 1
            for index2, row2 in tmp_df.loc[next_index:,:].iterrows():
                if row1["class_id"] == row2["class_id"]:
                    boxA = [row1['x_min'], row1['y_min'], row1['x_max'], row1['y_max']]
                    boxB = [row2['x_min'], row2['y_min'], row2['x_max'], row2['y_max']]
                    iou = bb_iou(boxA, boxB)
                    if iou > threshold:
                        if row1["index"] not in duplicate:
                            duplicate[row1["index"]] = []
                        duplicate[row1["index"]].append(row2["index"])
 
    remove_keys = []
    for k in duplicate:
        for i in duplicate[k]:
            if i in duplicate:
                for id in duplicate[i]:
                    if id not in duplicate[k]:
                        duplicate[k].append(id)
                if i not in remove_keys:
                    remove_keys.append(i)
    for i in remove_keys:
        del duplicate[i]
 
    rows = []
    removed_index = []
    for k in duplicate:
        row = tmp_df[tmp_df['index'] == k].iloc[0]
        X_min = [row['x_min']]
        X_max = [row['x_max']]
        Y_min = [row['y_min']]
        Y_max = [row['y_max']]
        removed_index.append(k)
        for i in duplicate[k]:
            removed_index.append(i)
            row = tmp_df[tmp_df['index'] == i].iloc[0]
            X_min.append(row['x_min'])
            X_max.append(row['x_max'])
            Y_min.append(row['y_min'])
            Y_max.append(row['y_max'])
        X_min_avg = sum(X_min) / len(X_min)
        X_max_avg = sum(X_max) / len(X_max)
        Y_min_avg = sum(Y_min) / len(Y_min)
        Y_max_avg = sum(Y_max) / len(Y_max)
        new_row = [row['image_id'], row['class_name'], row['class_id'], X_min_avg, Y_min_avg, X_max_avg, Y_max_avg, row['width'], row['height']]
        rows.append(new_row)
 
    for index, row in tmp_df.iterrows():
        if row['index'] not in removed_index:
            new_row = [row['image_id'], row['class_name'], row['class_id'], row['x_min'], row['y_min'], row['x_max'], row['y_max'], row['width'], row['height']]
            rows.append(new_row)
 
    new_df = pd.DataFrame(rows, columns =['image_id', 'class_name', 'class_id', 'x_min', 'y_min', 'x_max', 'y_max', 'width', 'height'])
    return new_df
 
 
images_lst = df_train.groupby('image_id').apply(list).reset_index()['image_id'].tolist()
images_dir = '/content/vinbigdata/train/'
 
# Add class_id=14 and class_id=15
frames = []
for img_id in images_lst:
    sub_df = df_train[df_train['image_id'] == img_id].reset_index()
    sub_df = averageCoordinates(sub_df,  0.5)
    frames.append(sub_df)
    rows = []
    if int(sub_df["class_id"][0]) == 14:
        rows.append("14 0.5 0.5 1.0 1.0")
    else:
        for index, row in sub_df.iterrows():
            w = int(row["width"])
            h = int(row["height"])
            cx = (int(row["x_min"]) + int(row["x_max"])) / 2 / w
            cy = (int(row["y_min"]) + int(row["y_max"])) / 2 / h
            bw = (int(row["x_max"]) - int(row["x_min"])) / w
            bh = (int(row["y_max"]) - int(row["y_min"])) / h
            row = [str(row["class_id"]), str(cx), str(cy), str(bw), str(bh)]
            rows.append(" ".join(row))
        rows.append("15 0.5 0.5 1.0 1.0")
    f = open(images_dir + img_id + ".txt", "w")
    f.write("\n".join(rows))
    f.close()
 
new_df = pd.concat(frames)
new_df.to_csv('/content/new_train.csv', index=False)

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split

df_train = pd.read_csv('/content/new_train.csv')
gkf  = GroupKFold(n_splits = 5)
df_train['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(df_train, groups = df_train.image_id.tolist())):
    df_train.loc[val_idx, 'fold'] = fold
df_train.head()

,image_id,class_name,class_id,x_min,y_min,x_max,y_max,width,height,fold
0,0005e8e3701dfb1dd93d53e2ff537b6e,Lung Opacity,7,902.5,585.0,1204.0,889.0,3072,3072,2
1,0005e8e3701dfb1dd93d53e2ff537b6e,Nodule/Mass,8,932.0,567.0,1197.0,896.0,3072,3072,2
2,0005e8e3701dfb1dd93d53e2ff537b6e,Infiltration,6,900.0,587.0,1205.0,888.0,3072,3072,2
3,0005e8e3701dfb1dd93d53e2ff537b6e,Consolidation,4,932.0,567.0,1197.0,896.0,3072,3072,2
4,0007d316f756b3fa0baea2ff514ce945,Aortic enlargement,0,1255.0,1030.0,1490.0,1261.0,2304,2880,0


In [ ]:
val = df_train[df_train.fold==fold]
train = df_train[df_train.fold!=fold]

In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
 
# df_train = pd.read_csv('/content/new_train.csv')
# train, val = train_test_split(df_train, test_size=0.2)
 
train_images_lst = train.groupby('image_id').apply(list).reset_index()['image_id'].tolist()
val_images_lst = val.groupby('image_id').apply(list).reset_index()['image_id'].tolist()
 
 
def createImagesTxt(_images, filepath):
    images_dir = '/content/vinbigdata/train/'
    rows = []
    for img_id in _images:
        rows.append(images_dir + img_id + '.png')
    f = open(filepath, "w")
    f.write("\n".join(rows))
    f.close()
 
 
train_path = '/content/train.txt'
val_path = '/content/val.txt'
createImagesTxt(train_images_lst, train_path)
createImagesTxt(val_images_lst, val_path)

In [ ]:
val.describe()

,class_id,x_min,y_min,x_max,y_max,width,height,fold
count,4763.000000,4763.000000,4763.000000,4763.000000,4763.000000,4763.000000,4763.000000,4763.0
mean,7.198824,1061.189131,1067.908461,1455.562916,1437.541955,2557.334033,2887.770523,4.0
std,4.384471,595.231426,575.916958,617.126164,607.811920,359.505511,269.500336,0.0
min,0.000000,0.000000,0.500000,128.000000,122.000000,1704.000000,1832.000000,4.0
25%,3.000000,564.500000,592.583333,941.000000,960.000000,2304.000000,2825.500000,4.0
50%,8.000000,1008.666667,927.000000,1484.000000,1383.333333,2540.000000,2880.000000,4.0
75%,11.000000,1509.833333,1503.250000,1936.000000,1937.000000,2832.500000,3072.000000,4.0
max,13.000000,2848.000000,2933.000000,3127.000000,3094.500000,3320.000000,3408.000000,4.0


In [ ]:
train.describe()

,class_id,x_min,y_min,x_max,y_max,width,height,fold
count,19055.000000,19055.000000,19055.000000,19055.000000,19055.000000,19055.000000,19055.000000,19055.000000
mean,7.254579,1048.869435,1075.378453,1439.954229,1448.394704,2517.488796,2867.902283,1.499921
std,4.333008,599.280243,577.465671,621.189916,605.249581,338.584867,283.680265,1.118040
min,0.000000,0.000000,0.000000,68.000000,103.000000,1572.000000,1712.000000,0.000000
25%,3.000000,548.000000,605.000000,919.000000,979.000000,2304.000000,2738.000000,0.500000
50%,8.000000,987.000000,951.000000,1484.000000,1416.000000,2488.000000,2880.000000,1.000000
75%,11.000000,1510.000000,1497.000000,1934.000000,1919.416667,2757.000000,3072.000000,2.000000
max,13.000000,2934.000000,3098.000000,3056.000000,3384.000000,3320.000000,3408.000000,3.000000


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
 
df_train = pd.read_csv('/content/new_train.csv')
train, val = train_test_split(df_train, test_size=0.2)
 
train_images_lst = train.groupby('image_id').apply(list).reset_index()['image_id'].tolist()
val_images_lst = val.groupby('image_id').apply(list).reset_index()['image_id'].tolist()
 
 
def createImagesTxt(_images, filepath):
    images_dir = '/content/vinbigdata/train/'
    rows = []
    for img_id in _images:
        rows.append(images_dir + img_id + '.png')
    f = open(filepath, "w")
    f.write("\n".join(rows))
    f.close()
 
 
train_path = '/content/train.txt'
val_path = '/content/val.txt'
createImagesTxt(train_images_lst, train_path)
createImagesTxt(val_images_lst, val_path)

In [ ]:
!rm -f /content/config.yaml
!echo "train: /content/train.txt" >> /content/config.yaml
!echo "val: /content/val.txt" >> /content/config.yaml
!echo "nc: 16" >> /content/config.yaml
!echo "names: ['Aortic enlargement','Atelectasis','Calcification','Cardiomegaly','Consolidation','ILD','Infiltration','Lung Opacity','Nodule/Mass','Other lesion','Pleural effusion','Pleural thickening','Pneumothorax','Pulmonary fibrosis','No finding','Finding']" >> /content/config.yaml

In [ ]:
!rm -f /content/config.yaml
!echo "train: /content/train.txt" >> /content/config.yaml
!echo "val: /content/val.txt" >> /content/config.yaml
!echo "nc: 14" >> /content/config.yaml
!echo "names: ['Aortic enlargement','Atelectasis','Calcification','Cardiomegaly','Consolidation','ILD','Infiltration','Lung Opacity','Nodule/Mass','Other lesion','Pleural effusion','Pleural thickening','Pneumothorax','Pulmonary fibrosis']" >> /content/config.yaml

In [ ]:
%cd /content/drive/MyDrive/yolo200/yolov5/
!pip install -qr requirements.txt

/content/drive/MyDrive/yolo200/yolov5
     |████████████████████████████████| 645kB 9.3MB/s 


In [ ]:
!pip install wandb

     |████████████████████████████████| 2.1MB 7.9MB/s 
     |████████████████████████████████| 133kB 49.8MB/s 
     |████████████████████████████████| 102kB 12.1MB/s 
     |████████████████████████████████| 163kB 37.6MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=56b8a046c61730d9ee64d97083ea537b3c77a7489996925d074cef461f76b49d
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=d7ff32c7715954561dc0144fcbdc31d2f353b4347bf76235998cb375cf573e92
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [ ]:
import wandb
wandb.login(key='44f4774b01085df4610534309a809d870c5c575f')

wandb: Currently logged in as: tphuc (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!python train.py --img 640 --batch 16 --epochs 15 --data /content/config.yaml --weights yolov5x.pt

In [ ]:
!python train.py --resume

github: ⚠️ WARNING: code is out of date by 16 commits. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov5' to download latest.
Resuming training from ./runs/train/exp/weights/last.pt
YOLOv5 🚀 2021-3-29 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='/content/config.yaml', device='', entity=None, epochs=200, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=True, save_dir='runs/train/exp', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=16, upload_dataset=False, weights='./runs/train/exp/weights/last.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view 

In [ ]:
!zip -r /content/xray200.zip /content/drive/MyDrive/yolo200/yolov5/runs/train/

  adding: content/drive/MyDrive/yolo200/yolov5/runs/train/ (stored 0%)
  adding: content/drive/MyDrive/yolo200/yolov5/runs/train/exp/ (stored 0%)
  adding: content/drive/MyDrive/yolo200/yolov5/runs/train/exp/weights/ (stored 0%)
  adding: content/drive/MyDrive/yolo200/yolov5/runs/train/exp/weights/last.pt (deflated 8%)
  adding: content/drive/MyDrive/yolo200/yolov5/runs/train/exp/weights/best.pt (deflated 8%)
  adding: content/drive/MyDrive/yolo200/yolov5/runs/train/exp/labels_correlogram.jpg (deflated 34%)
  adding: content/drive/MyDrive/yolo200/yolov5/runs/train/exp/labels.jpg (deflated 21%)
  adding: content/drive/MyDrive/yolo200/yolov5/runs/train/exp/train_batch0.jpg (deflated 4%)
  adding: content/drive/MyDrive/yolo200/yolov5/runs/train/exp/train_batch1.jpg (deflated 3%)
  adding: content/drive/MyDrive/yolo200/yolov5/runs/train/exp/train_batch2.jpg (deflated 3%)
  adding: content/drive/MyDrive/yolo200/yolov5/runs/train/exp/events.out.tfevents.1616983319.baf2cadce8df.570.0 (deflate

In [ ]:
!zip -r /content/vinbigdata.zip /content/vinbigdata

In [ ]:
from shutil import copyfile
#copyfile('/content/xray0803_80.zip', '/content/drive/MyDrive/new/xray0803_80.zip')
copyfile('/content/drive/MyDrive/new/xray0803_80.zip','/content/xray0803_80.zip')

'/content/xray0803_80.zip'

In [ ]:
from shutil import copyfile
copyfile('/content/xray200.zip', '/content/drive/MyDrive/xray200.zip')
#copyfile('/content/drive/MyDrive/new/xray0803_95.zip','/content/xray0803_95.zip')

'/content/drive/MyDrive/xray200.zip'

In [ ]:
pwd

'/content/drive/My Drive/yolo200/yolov5'

In [ ]:
ls

data/       hubconf.py  __pycache__/      runs/     tutorial.ipynb  weights/
detect.py   LICENSE     README.md         test.py   utils/          yolov5x.pt
Dockerfile  models/     requirements.txt  train.py  wandb/


In [ ]:
cd /content/drive/MyDrive/yolo200/yolov5/runs/train/exp/

/content


In [ ]:
!unzip -q xray0803_95.zip

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 6294, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 6294 (delta 41), reused 59 (delta 24), pack-reused 6205
Receiving objects: 100% (6294/6294), 8.54 MiB | 29.95 MiB/s, done.
Resolving deltas: 100% (4300/4300), done.


In [ ]:
cd yolov5/

/content/yolov5


In [ ]:
!pip install -qr requirements.txt

     |████████████████████████████████| 645kB 26.7MB/s 


In [ ]:
!python train.py --img 640 --batch 16 --epochs 20 --data /content/config.yaml --weights yolov5x.pt

github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-91-g17b0f71 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='/content/config.yaml', device='', entity=None, epochs=20, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=16, upload_dataset=False, weights='yolov5x.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2021-05-16 01:28:57.000881: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened d